In [2]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [3]:
%matplotlib inline

from pprint import pprint
from zen_garden.postprocess.results import Results

from pathlib import Path
from os import chdir

In [4]:
#Set path
path = Path('/Users/davidbertschinger/Documents/GitHub/ZEN-garden/notebooks')
chdir(path)

In [5]:
# cost optimal solution
res_basic = Results("../data/outputs/01_mini_CCTS")

# cost optimal solution with existing capacities
res_existing = Results("../data/outputs/01_mini_CCTS_existing")

In [6]:
# Show the deafult analysis settings
pprint(res_basic.results[None]["analysis"])

{'dataset': '/Users/davidbertschinger/Documents/GitHub/ZEN-garden/data/01_mini_CCTS',
 'earliest_year_of_data': 1900,
 'folder_name_system_specification': 'system_specification',
 'folder_output': '/Users/davidbertschinger/Documents/GitHub/ZEN-garden/data/outputs',
 'header_data_inputs': {'set_capacity_types': 'capacity_type',
                        'set_carriers': 'carrier',
                        'set_conversion_technologies': 'technology',
                        'set_dependent_carriers': 'carrier',
                        'set_edges': 'edge',
                        'set_elements': 'element',
                        'set_input_carriers': 'carrier',
                        'set_lca_impact_categories': 'impact_category',
                        'set_location': 'location',
                        'set_n1_contingency': 'contingency',
                        'set_nodes': 'node',
                        'set_output_carriers': 'carrier',
                        'set_storage_technologies

In [7]:
res_basic.get_df('capacity', to_csv="capacity")
res_existing.get_df('capacity', to_csv="capacity2")

technology     capacity_type  location                     year
capture_amine  power          swiss_storage_CH             0        0.000000
                                                           1        0.000000
                                                           2        0.000000
                              waste_1_CH                   0       13.577313
                                                           1       22.528265
                                                                     ...    
truck_batch    power          waste_1_CH-waste_2_CH        1        0.000000
                                                           2        0.000000
                              waste_2_CH-swiss_storage_CH  0        0.000000
                                                           1        0.000000
                                                           2        1.513804
Name: capacity, Length: 108, dtype: float64

In [8]:
#Get capacity table
df = pd.read_csv('capacity.csv')
df2= pd.read_csv('capacity2.csv')

In [9]:
df

,technology,capacity_type,location,year,capacity
0,capture_amine,power,swiss_storage_CH,0,0.000000
1,capture_amine,power,swiss_storage_CH,1,0.000000
2,capture_amine,power,swiss_storage_CH,2,0.000000
3,capture_amine,power,waste_1_CH,0,13.577313
4,capture_amine,power,waste_1_CH,1,22.528265
...,...,...,...,...,...
103,truck_batch,power,waste_1_CH-waste_2_CH,1,0.000000
104,truck_batch,power,waste_1_CH-waste_2_CH,2,0.000000
105,truck_batch,power,waste_2_CH-swiss_storage_CH,0,0.000000
106,truck_batch,power,waste_2_CH-swiss_storage_CH,1,0.000000


In [10]:
# Subtract 'capacity' values in df from df2
result_df = df2.copy()  # Make a copy of df2 to avoid modifying the original DataFrame
result_df['capacity'] = df2['capacity'] - df['capacity']
result_df = result_df[result_df['capacity'] != 0]
result_df

,technology,capacity_type,location,year,capacity


In [22]:
#Find unique Technologies
technology = df['technology'].unique()
technology

array(['capture_amine', 'compression_16_110bar', 'condition_110bar',
       'condition_16bar', 'decompression_110_16bar',
       'decompression_110bar_ambient', 'emergency_storage', 'emitter_wte',
       'heat_pump', 'permanent_storage', 'flowline', 'truck_batch'],
      dtype=object)

In [24]:
#Create existing capacity files

for i in range(len(technology)):
     
    #Get table of one technology 
    tec = technology[i]
    x = df[df['technology'] == tec]
    
    #Set path
    parent_path = Path('/Users/davidbertschinger/Documents/GitHub/ZEN-garden/data/01_mini_CCTS_existing')
    path = [file for file in Path(parent_path).rglob(tec)]
    path = path[0]
    filepath = Path(str(path)+'/capacity_existing.csv')
    
    #Clean up table
    x = x.drop(columns='capacity_type')
    x = x.drop(columns='technology')
    x = x[x['capacity'] != 0]
    x = x.rename(columns={'year': 'year_construction'})
    x = x.rename(columns={'capacity': 'capacity_existing'})
    x['year_construction'] = x['year_construction'] + 2025 
    if not x['capacity_existing'].empty:
        x['capacity_existing'] = x['capacity_existing'].diff().fillna(x['capacity_existing'].iat[0]).clip(lower=0)
    
    if  tec == 'biomethane_boiler':
        x['capacity_existing'] = x['capacity_existing'] / 1000
    
    #clean up table, differentiate between node and edge
    if path.parent.name == 'set_transport_technologies':
        x = x.rename(columns={'location': 'edge'})

    else:
        x = x.rename(columns={'location': 'node'})

    #Safe
    x.to_csv(filepath, index=False)

In [19]:
tec = technology[2]
x = df[df['technology'] == tec]

#Clean up table
x = x.drop(columns='capacity_type')
x = x.drop(columns='technology')
x = x[x['capacity'] != 0]
x = x.rename(columns={'year': 'year_construction'})
x = x.rename(columns={'capacity': 'capacity_existing'})
x['year_construction'] = x['year_construction'] + 2025 
x

,location,year_construction,capacity_existing


In [30]:
tec = technology[10]
x = df[df['technology'] == tec]

#Clean up table
x = x.drop(columns='capacity_type')
x = x.drop(columns='technology')
x = x[x['capacity'] != 0]

x

,location,year,capacity
12300,cement_1_CH,0,57.941061
12301,cement_1_CH,1,57.941061
12302,cement_1_CH,2,57.941061
12303,cement_1_CH,3,57.941061
12304,cement_1_CH,4,57.941061
...,...,...,...
12445,cement_6_CH,20,40.444902
12446,cement_6_CH,21,40.444902
12447,cement_6_CH,22,40.444902
12448,cement_6_CH,23,40.444902


In [35]:
x['increments'] = x['capacity_existing'].diff().fillna(0).clip(lower=0)

,technology,capacity_type,location,year,capacity
176775,flowline,power,basel_export_CH-cement_1_CH,0,0.0
176776,flowline,power,basel_export_CH-cement_1_CH,1,0.0
176777,flowline,power,basel_export_CH-cement_1_CH,2,0.0
176778,flowline,power,basel_export_CH-cement_1_CH,3,0.0
176779,flowline,power,basel_export_CH-cement_1_CH,4,0.0
...,...,...,...,...,...
216370,flowline,power,zeebrugge_BE-wintershall_dea_luna_NO,20,0.0
216371,flowline,power,zeebrugge_BE-wintershall_dea_luna_NO,21,0.0
216372,flowline,power,zeebrugge_BE-wintershall_dea_luna_NO,22,0.0
216373,flowline,power,zeebrugge_BE-wintershall_dea_luna_NO,23,0.0


In [26]:
x


TypeError: unsupported operand type(s) for -: 'str' and 'str'

NameError: name 'column' is not defined